In [ ]:
import DataMap
from ipyleaflet import basemaps, basemap_to_tiles
import panel as pn, datetime as dt, os, pandas as pd, hvplot.pandas, geopandas, json, math
pn.extension(sizing_mode="stretch_width")

In [ ]:
# -------------------------------------------------- Constant Variables --------------------------------------------------

# Assign names for map's layer types.
topography_data = "Topography"
bathymetry_kayak_data = "Nearshore Bathymetry - Kayak"
bathymetry_watercraft_data = "Nearshore Bathymetry - Personal Watercraft"
grainsize_data = "Surface-Sediment Grain-Size Distributions"
basemap_data = "Basemap"

elwha_data_types = [topography_data]
# [topography_data, grainsize_data]
# [topography_data, bathymetry_kayak_data, bathymetry_watercraft_data, grainsize_data]

elwha_basemap_options = {
  "Default": basemaps.OpenStreetMap.Mapnik,
  "Satellite": basemaps.Esri.WorldImagery,
  "Topographic": basemaps.OpenTopoMap,
  "Black & White": basemaps.Stamen.Toner,
  "Dark": basemaps.CartoDB.DarkMatter
}

elwha_map = DataMap(
  map_center = [48.148, -123.553],
  basemap_options = elwha_basemap_options,
  legend = {
    "name": "Types of Data",
    "colors": {
      topography_data: "red",
      bathymetry_kayak_data: "yellow",
      bathymetry_watercraft_data: "green",
      grainsize_data: "#975411"
    }
  }
)

# -------------------------------------------------- Elwha Topo-Bathy Data Widgets --------------------------------------------------

basemap_select = pn.widgets.Select(name="Basemap", options=list(elwha_basemap_options.keys()))
elwha_data_multi_choice = pn.widgets.MultiChoice(name="Type of Data", value=[grainsize_data], options=elwha_data_types, solid=False)
data_date_range_slider = pn.widgets.DateRangeSlider(
  name="Data Collection Range",
  start=dt.datetime(2010, 9, 5), end=dt.datetime.utcnow(),
  value=(dt.datetime(2018, 1, 1), dt.datetime(2019, 1, 1)),
  bar_color="#0066cc"
)

# -------------------------------------------------- Helper Functions --------------------------------------------------

# Gets the point_style based on the data type.
def get_data_type_point_style(data_type):
  if data_type == topography_data:
    return {"color": "red", "opacity": 0.5, "radius": 0}     # simple dot for topography data since there's many datapoints for one latitude-longitude coordinate
  elif data_type == bathymetry_kayak_data:
    return {"color": "yellow", "radius": 0}
  elif data_type == bathymetry_watercraft_data:
    return {"color": "green", "radius": 0}
  elif data_type == grainsize_data:
    return {"color": "#975411", "opacity": 0.5, "fillColor": "#975411", "fillOpacity": 0.3, "radius": 8, "weight": 1, "dashArray": 2}
  return {}

# Gets the hover_style based on the data type.
def get_data_type_hover_style(data_type):
  if data_type == topography_data:
    return {"color": "#0066cc"}
  return {"color": "#0066cc", "fillColor": "#0066cc", "weight": 3}

# Filters data based on what data type(s) and time range that the user selects.
def filter_data_on_map(event):
  # selected_data_types = elwha_data_multi_choice.value
  # (selected_start_date, selected_end_date) = data_date_range_slider.value
  # start_month, start_day, start_year = selected_start_date.month, selected_start_date.day, selected_start_date.year
  # end_month, end_day, end_year = selected_end_date.month, selected_end_date.day, selected_end_date.year
  # for data_type in elwha_data_types:
  #   for layer in elwha_map_layers[data_type]:
  #     # Add the data layer if it isn't in map yet.
  #     if (data_type in selected_data_types) and (layer not in elwha_map.layers):
  #       # print("display", layer.name)
  #       elwha_map.add_layer(layer)
  #     # Else remove the data layer if user didn't select to display it and its data is in the map.
  #     elif (data_type not in selected_data_types) and (layer in elwha_map.layers):
  #       # print("remove", layer.name)
  #       elwha_map.remove_layer(layer)
  print("filter", event)

# Displays the tile layer of the newly selected basemap.
def update_tile_layers(event):
  for basemap_name in elwha_basemap_options.keys():
    basemap_tile_layer = elwha_map.all_layers[basemap_name]
    # # Remove the filtering data callback function (only need to execute it once at the beginning of loading the default basemap).
    # basemap_tile_layer.on_load(callback=filter_data_on_map, remove=True)
    if basemap_tile_layer.name == basemap_select.value: basemap_tile_layer.visible = True
    else: basemap_tile_layer.visible = False

# -------------------------------------------------- Callbacks --------------------------------------------------

# Filter data whenever the selected data type(s) or date range change.
elwha_data_multi_choice.param.watch(filter_data_on_map, "value")
data_date_range_slider.param.watch(filter_data_on_map, "value")

# Update the visibility of tile layers whenever a different basemap is selected.
basemap_select.param.watch(update_tile_layers, "value")

# -------------------------------------------------- Add Map Layers --------------------------------------------------

## Plot collected data as ipyleaflet layers.
for data_type in elwha_data_types:
  # Convert each data file into GeoJSON.
  data_files = os.listdir("data/Elwha/" + data_type)
  for file in data_files:
    print("Loading data from " + file + "...")
    # Determine popup content based on different types of data.
    popup_content = {}
    if data_type == grainsize_data:
      popup_content = {
        "Date & Time Collected": [
          "Date Collected",
          {
            "Time (GMT)": "GMT",      # for grainsize data before July 2018
            "Time_GMT": "GMT"         # for grainsize data at and after July 2018
          }
        ],
        "Sample Type": ["Sample Type"],
        "Weight": ["Wt. percent in -2.00 phi bin", ("%")],
        "Gravel": ["Percent Gravel", ("%")],
        "Sand": ["Percent Sand", ("%")],
        "Silt": ["Percent Silt", ("%")],
        "Clay": ["Percent Clay", ("%")],
        "Mud": ["Percent Mud", ("%")]
      }
    elif (data_type == topography_data) or (data_type == bathymetry_kayak_data) or (data_type == bathymetry_watercraft_data):
      popup_content = {
        "Date & Time Collected": [
          {
            "Survey_Date": "",        # for topo-bathy data before July 2018
            "datetime_utc": "UTC"     # for topo-bathy data at and after July 2018
          }
        ],
        "Orthometric Height": [
          {
            "Ortho_Ht_m": "meters",
            "Ortho_ht_m": "meters",
            "ortho_ht_m": "meters"
          },
        ]
      }
    # Create and add GeoJSON layer to map.
    elwha_map.create_geojson_layer(
      data_path = "data/Elwha/" + data_type + "/" + file,
      layer_name = file,
      popup_content = popup_content,
      longitude_col_name = ["Longitude", "longitude", "Longitude (deg. E)"],
      latitude_col_name = ["Latitude", "latitude", "Latitude (deg. N)"],
      point_style = get_data_type_point_style(data_type),
      hover_style = get_data_type_hover_style(data_type)
    )
    
    # Group data from the same location into clusters.
    # marker_cluster, cluster_location = MarkerCluster(name="Clusters"), None
    # for (index, row) in dataframe.iterrows():
    #   marker_lat, marker_long = get_latitude(row), get_longitude(row)
    #   cluster_lat, cluster_long = round(marker_lat, 4), round(marker_long, 4)
    #   marker = Marker(location=[marker_lat, marker_long], visible=
    # False)
    #   # Add new marker cluster to map when the coordinates don't belong to the current cluster.
    #   if (cluster_location is not None) and (not math.isclose(cluster_lat, cluster_location[0]) or not math.isclose(cluster_long, cluster_location[1])):
    #     # print("new cluster at " + str(cluster_location) + " is added to the map")
    #     elwha_map.add_layer(marker_cluster)
    #     marker_cluster, cluster_location = MarkerCluster(name="Clusters"), [cluster_lat, cluster_long]
    #   # Else add marker to the current cluster.
    #   else:
    #     if cluster_location is None: cluster_location = [cluster_lat, cluster_long]
    #     # print("marker at [" + str(marker_lat) + ", " + str(marker_long) + "] is added to the current cluster at " + str(cluster_location))
    #     marker_cluster.markers += (marker,)
    # elwha_map.add_layer(marker_cluster)     # need to add last marker cluster because last marker/row in dataframe will be added to last cluster but for loop never adds the cluster to the map

## Create Elwha component.
elwha_component = pn.Column(elwha_map, basemap_select, elwha_data_multi_choice, data_date_range_slider)

In [ ]:
## Add components to serve on app.
data_visualizer = pn.Column(elwha_component)
data_visualizer.servable()